In [1]:
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from qcodes.dataset.plotting import plot_dataset,plot_by_id
from qcodes.logger.logger import start_all_logging
# from qcodes.plots.pyqtgraph import QtPlot
from qcodes.loops import Loop
from time import sleep
from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import qcodes as qc
import numpy as np
import datetime
import time
import json
import sys
import os


In [2]:
sys.path.append('C:\QCodes_Driver_And_Scripts\Drivers')

from qcodes.instrument_drivers.yokogawa.GS200 import GS200

from Agilent_E8257D import E8257D
from RS_SGS_100A import SGS100A
from RS_SMB_100A import SMB100A
from HP3245A import HP3245A
from RS_FSV import RS_FSV
from VNA_RS import VNA

try:
    rf_source=E8257D('rf_source','TCPIP0::192.168.10.70::inst0::INSTR')

except KeyError as er:

    E8257D.close_all()
    rf_source=E8257D('rf_source','TCPIP0::192.168.10.70::inst0::INSTR')
    
try:

    lo_source=SMB100A('lo_pump','TCPIP0::192.168.10.6::inst0::INSTR')

except KeyError as er:

    SMB100A.close_all()
    lo_source=SMB100A('lo_pump','TCPIP0::192.168.10.6::inst0::INSTR')

try:
    FSV = RS_FSV('rs_fsv', 'TCPIP0::192.168.10.66::inst0::INSTR')

except KeyError as er:


    RS_FSV.close_all() #Disconnect and irreversibly tear down the instrument
    FSV =  RS_FSV('rs_fsv', 'TCPIP0::192.168.10.66::inst0::INSTR')


Connected to: Agilent Technologies E8257D (serial:US45470698, firmware:C.06.28) in 0.31s
Connected to: Rohde&Schwarz SMB100A (serial:1406.6000k03/177959, firmware:3.1.18.2-3.01.203.32) in 0.03s
Connected to: Rohde&Schwarz FSV3030 (serial:1330.5000K30/101857, firmware:1.80) in 0.03s


In [3]:
def measure(exp, station):

    meas = qc.Measurement(exp=exp, station=station)
    meas.register_parameter(FSV.freq_axis)
    meas.register_parameter(FSV.spectrum)

    with meas.run() as datasaver:
        
        run_id=datasaver.dataset.run_id
        qc.load_by_run_spec(captured_run_id=run_id).add_metadata('parameter_snap',
                                 json.dumps(parameter_snap))

        get_s = 1.e-3*10**(FSV.spectrum.get()/10)
        get_c = FSV.freq_axis.get()
        
        sleep(0.01)

        datasaver.add_result((FSV.freq_axis, get_c),(FSV.spectrum,get_s))

In [4]:
start_all_logging()

# Create a station
station = qc.Station()
station.add_component(FSV)
station.add_component(lo_source)
station.add_component(rf_source)

station.snapshot()
station.components

# Experiment details
user='DF'
date=datetime.datetime.today().strftime('%Y_%m_%d')
description='FSV_single_trace'
database_name = date+"_"+user+"_"+description


Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\nicolas.ROCH\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\nicolas.ROCH\.qcodes\logs\220922-51904-qcodes.log


In [5]:
script_dir=os.path.dirname(os.path.abspath("__file__"))
data_dir=os.path.join('C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\'+date)

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

data_dir=data_dir +'\\'+description

try :
    os.mkdir(data_dir)
except FileExistsError:
    pass

In [6]:
qc.initialise_or_create_database_at(data_dir+'\\'+database_name)
qc.config.core.db_location

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1002.70it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.11it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v8 -> v9: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 501.35it/s]


'C:\\QCodes_Driver_And_Scripts\\Data_QCodes\\Dorian\\2022_09_22\\FSV_single_trace\\2022_09_22_DF_FSV_single_trace'

In [29]:
current = 0. #mA

# lo source

lo_freq  = 2.96 #GHz
lo_power = 5   #dBm

# rf source

rf_freq  = 0.04 #GHz
rf_power = 0  #dBm

# Spectral Analyser parameters

FSV_rbw    = 2*500*1.e3 #Hz
FSV_vbw    = 1.0*1.e3 #Hz
FSV_ref    = 10      #dB
FSV_avg    = 0
FSV_att    = 0        #dB
FSV_start  = 5.825    #GHz
FSV_stop   = 6.01    #GHz
FSV_npt    = 1024

In [30]:
FSV.res_BW(FSV_rbw)
FSV.video_BW(FSV_vbw)
FSV.f_start(FSV_start*1.e9)
FSV.f_stop(FSV_stop*1.e9)
FSV.averages(FSV_avg)
FSV.n_points(FSV_npt)
FSV.ref_level(FSV_ref)
FSV.input_att(FSV_att)

FSV.write('SWE:TYPE AUTO')
FSV.detector('rms')

lo_source.frequency(lo_freq*1.e9)
lo_source.power(lo_power)

rf_source.frequency(rf_freq*1.e9)
rf_source.power(rf_power)

lo_source.status('on')
rf_source.status('ON')

parameter_snap={}

parameter_snap['lo_source'] = {'power':lo_power,'frequency':lo_freq}
parameter_snap['rf_source'] = {'power':rf_power,'frequency':rf_freq}

parameter_snap['FSV']={'start_freq':FSV_start,'stop_freq':FSV_stop,'average':FSV_avg,'res_bandwidth':FSV_rbw,'vid_bandwidth':FSV_vbw,
                  'Npoints':int(FSV.n_points()),'ref':FSV_ref,'att':FSV_att}

# Load experiment
exp_name = 'FSV_single_trace'
sample_name = 'ssb8012A_lsb'

exp=qc.load_or_create_experiment(experiment_name=exp_name,
                          sample_name=sample_name)

measure(exp=exp, station=station)

# Shutdown devices

lo_source.status('off')
rf_source.status('OFF')

Starting experimental run with id: 12. 
